In [ ]:
# Load necessary packages
import jax
import jax.numpy as jnp
import numpy as np
import optax
import matplotlib.pyplot as plt

# User-defined
from ANN_1DOF import Damped_MLP, Damped_LNN
from helpers import train_test_data_old, train_test_data, compare_sols, format_to_LNN

from jax import config
config.update("jax_enable_x64", True)

#### Generate Training Data

In [ ]:
# Generate continuation data
old_old_train_dataset, old_old_test_dataset, old_info = train_test_data_old(
    save_file='data/data.pkl', 
    split_size=0.2, 
    file_name='contparameters.json', 
    min_force_amp=0.1, 
    max_force_amp=1.0, 
    num=2, 
    phase_ratio=0.5, 
    damping=0.05
)

In [ ]:
# Generate continuation data
old_train_dataset, old_test_dataset, info = train_test_data(
    save_file='data/data.pkl', 
    split_size=0.2, 
    file_name='contparameters.json', 
    min_force_amp=0.1, 
    max_force_amp=1.0, 
    num=2, 
    phase_ratio=0.5, 
    damping=0.05
)

#### Format Dataset for LNN Code

In [ ]:
old_old_train_dataset['x'].shape, old_old_train_dataset['dx'].shape, old_old_train_dataset['ddx'].shape, old_old_train_dataset['t'].shape, old_old_train_dataset['f'].shape

In [ ]:
old_train_dataset['x'].shape, old_train_dataset['dx'].shape, old_train_dataset['ddx'].shape, old_train_dataset['t'].shape, old_train_dataset['f'].shape

In [ ]:
# Plot figures
fig, ax = plt.subplots(4, 1, figsize=(10, 10))

ax[0].plot(old_old_train_dataset['x'][:, 0], label='old')
ax[0].plot(old_train_dataset['x'][:, 0])
ax[1].plot(old_old_train_dataset['dx'][:, 0], label='old')
ax[1].plot(old_train_dataset['dx'][:, 0])
ax[2].plot(old_old_train_dataset['ddx'][:, 0], label='old')
ax[2].plot(old_train_dataset['ddx'][:, 0])
ax[3].plot(old_old_train_dataset['f'][:, 0], label='old')
ax[3].plot(old_train_dataset['f'][:, 0])

plt.legend()

In [ ]:
train_dataset, test_dataset, info = format_to_LNN(old_train_dataset, old_test_dataset, info)

#### LNN Training

In [ ]:
settings = {
    'name': 'ANN_LNN_Damped_Test',
    'lag_units': 64,
    'damp_units': 16,
    'layers': 3,
    'input_shape': 2,
    'train_batch_size': 128,
    'test_batch_size': 64,
    'shuffle': True,
    'seed': 0
    }

phy_sys = {
    'M': info['M'],
    'K': info['K'],
    'C': info['C'],
    'NL': info['NL'],
}

lr = 1e-03
optimizer = optax.adam(lr)
a = Damped_LNN(Damped_MLP, optimizer, settings, info, phy_sys)
a.gather()

In [ ]:
# # Load previous results, if any
# prev_results = a.load_model('results/ANN_Damped_100epochs/model.pkl')
# results = prev_results

In [ ]:
# Start training LNN
results = a.train(train_dataset, test_dataset, results=None, epochs=100, show_every=10)

In [ ]:
# Save results
a.save_model(results, 'results/ANN_Damped_100epochs')

#### Examine Data Scope

In [23]:
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

with open('./data/data.pkl', 'rb') as f:
   data = pickle.load(f)

In [30]:
datas = []

for k, v in data.items():
    pose = data[k]["pose"]
    vel = data[k]["vel"]
    acc = data[k]["acc"]
    time = data[k]["time"]
    force = data[k]["force"]
    
    # Create train & test split with equal split for each forcing amplitude
    pose_train, pose_test, vel_train, vel_test, acc_train, acc_test, time_train, time_test, force_train, force_test = train_test_split(pose, vel, acc, time, force, test_size=0.2, random_state=42, shuffle=True)
    
    # Append
    datas.append((pose_train, pose_test, vel_train, vel_test, acc_train, acc_test, time_train, time_test, force_train, force_test))

In [53]:
datas[0][0].shape, datas[1][0].shape

((240, 142), (240, 189))

In [54]:
x_train = np.array([])

for i in range(2):
    x_train = np.append(x_train, datas[i][0], axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [48]:
for cond in datas:
    print(len(cond))

10
10


In [24]:
# Store ML data
x_train = np.array([])
dx_train = np.array([])
ddx_train = np.array([])
t_train = np.array([])
f_train = np.array([])
x_test = np.array([])
dx_test = np.array([])
ddx_test = np.array([])
t_test = np.array([])
f_test = np.array([])

# Loop over data
for k, v in data.items():
    pose = data[k]["pose"]
    vel = data[k]["vel"]
    acc = data[k]["acc"]
    time = data[k]["time"]
    force = data[k]["force"]
    T = data[k]["T"]
    
    # Create train & test split with equal split for each forcing amplitude
    pose_train, pose_test, vel_train, vel_test, acc_train, acc_test, time_train, time_test, force_train, force_test = train_test_split(pose, vel, acc, time, force, test_size=0.2, random_state=42, shuffle=True)
    
    # Reshape array
    x_train = x_train.reshape(pose_train.shape[0], -1)
    dx_train = dx_train.reshape(vel_train.shape[0], -1)
    ddx_train = ddx_train.reshape(acc_train.shape[0], -1)
    t_train = t_train.reshape(time_train.shape[0], -1)
    f_train = f_train.reshape(force_train.shape[0], -1)
    x_test = x_test.reshape(pose_test.shape[0], -1)
    dx_test = dx_test.reshape(vel_test.shape[0], -1)
    ddx_test = ddx_test.reshape(acc_test.shape[0], -1)
    t_test = t_test.reshape(time_test.shape[0], -1)
    f_test = f_test.reshape(force_test.shape[0], -1)
    
    # Collect into 1 array
    x_train = np.append(x_train, pose_train, axis=1)
    dx_train = np.append(dx_train, vel_train, axis=1)
    ddx_train = np.append(ddx_train, acc_train, axis=1)
    t_train = np.append(t_train, time_train, axis=1)
    f_train = np.append(f_train, force_train, axis=1)
    x_test = np.append(x_test, pose_test, axis=1)
    dx_test = np.append(dx_test, vel_test, axis=1)
    ddx_test = np.append(ddx_test, acc_test, axis=1)
    t_test = np.append(t_test, time_test, axis=1)
    f_test = np.append(f_test, force_test, axis=1)

In [25]:
pose_train

array([[ 0.17431091,  0.17438371,  0.17451456, ..., -0.01997459,
        -0.01799587, -0.01615657],
       [-0.11353207, -0.1136154 , -0.11373283, ...,  0.01041456,
         0.00942447,  0.0084974 ],
       [ 0.21807566,  0.21817446,  0.21834333, ..., -0.02415264,
        -0.02176978, -0.01955323],
       ...,
       [-0.23323835, -0.23335886, -0.23356788, ...,  0.02340486,
         0.02113316,  0.01901392],
       [ 0.17463602,  0.17470852,  0.17487472, ..., -0.01704675,
        -0.01540376, -0.01386912],
       [-0.24668315, -0.2468051 , -0.24702306, ...,  0.02487432,
         0.02245664,  0.02020177]])

### Examine results

In [ ]:
# Plotting the corresponding lagrangian and damping function for each output in the test dataset
pred_acc_damped, pred_energy_damped = a._predict(results)
q, q_d = jnp.split(test_dataset[0], 2, axis=-1)
n = 301
Lnn, Dnn = pred_energy_damped(q, q_d)

In [ ]:
# Comparing accelerations
F = test_dataset[1].reshape(q.shape[0], -1)
q_dd = pred_acc_damped(test_dataset[0], F)[:, -1]

In [ ]:
lim1, lim2 = info['qmax'], info['qdmax']

qa, qda = jnp.linspace(-lim1, lim1, 100), jnp.linspace(-lim2, lim2, 100)
qaa, qdaa = jnp.meshgrid(qa, qda)

# Get all energy functions here
L, D = jax.vmap(pred_energy_damped)(qaa.reshape(-1,1,1), qdaa.reshape(-1,1,1))
Lagrange_analy = 0.5*phy_sys['M']*qdaa**2 - 0.5*phy_sys['K']*qaa**2 - 0.25*phy_sys['NL']*qaa**4
Dissipation_analy = 0.5*phy_sys['C']*qdaa**2

In [ ]:
fig = plt.figure(figsize=(15,12), tight_layout=True)

ax = fig.add_subplot(221, projection='3d')
m = ax.plot_surface(qaa, qdaa, L.reshape(qaa.shape), alpha=0.5, cmap='plasma')
ax.contour3D(qaa, qdaa, L.reshape(qaa.shape), cmap='binary')
ax.set_xlabel('q')
ax.set_ylabel(r'$\dot{q}$')
ax.set_zlabel(r'$\mathcal{L}_{NN}$', fontsize=16, labelpad=3)
ax.set_title('Lagrangian prediction - overall phase space')
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

ax = fig.add_subplot(222, projection = '3d')
m = ax.plot_surface(qaa, qdaa, Lagrange_analy.reshape(qaa.shape), alpha=0.5, cmap='plasma')
ax.contour3D(qaa, qdaa, Lagrange_analy.reshape(qaa.shape), cmap='binary')
ax.set_title('Analytical Lagrangian')
ax.set_xlabel(r'$q \ (m)$', fontsize=12)
ax.set_ylabel(r'$\dot{q} \ (m \ s^{-1})$ ', fontsize=12)
ax.set_zlabel(r'L (J)', fontsize=14, labelpad=2)
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

ax = fig.add_subplot(223, projection='3d')
m = ax.plot_surface(qaa, qdaa, D.reshape(qaa.shape), alpha=0.7, cmap='magma')
ax.contour3D(qaa, qdaa, D.reshape(qaa.shape), cmap='binary')
ax.set_xlabel('q')
ax.set_ylabel(r'$\dot{q}$')
ax.set_zlabel(r'$\mathcal{D}_{NN}$', fontsize=16, labelpad=3)
ax.set_title('Dissipation prediction - overall phase space')
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

ax = fig.add_subplot(224, projection = '3d')
m = ax.plot_surface(qaa, qdaa, Dissipation_analy.reshape(qaa.shape), alpha=0.7, cmap='magma')
ax.contour3D(qaa, qdaa, Dissipation_analy.reshape(qaa.shape), cmap='binary')
ax.set_title('Analytical Dissipation')
ax.set_xlabel(r'$q \ (m)$', fontsize=12)
ax.set_ylabel(r'$\dot{q} \ (m \ s^{-1})$ ', fontsize=12)
ax.set_zlabel(r'D (J)', fontsize=14, labelpad=2)
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

#### Compare Analytical vs LNN FRCs

In [ ]:
compare_sols(anal_file='data_old/FRF6', lnn_file='data_LNN_old/FRF6')

In [ ]:
for i in range(1, 11, 1):
    compare_sols(anal_file=f'data_old/FRF{i}', lnn_file=f'data_LNN_old/FRF{i}')